In [44]:
import pandas as pd
import numpy as np

In [45]:
flight_details= pd.read_csv("/PNRB-ZZ-20231208_062017.csv")
passenger= pd.read_csv("/PNRP-ZZ-20231208_111136.csv")
schedule=pd.read_csv("/SCH-ZZ-20231208_035117.csv")
# flight_details

In [46]:
#input

specific_schedule_id= "SCH-ZZ-9117133"
specific_date="20240403"

In [47]:
# segment score and pax score input

external_multiplier_seg = 100
external_multiplier_pax = 50

In [48]:
#ssr cost input and loyalty cost

tags_cost ={'INFT' : 200, 'WCHR' : 200, 'WCHS' : 200, 'WCHC' : 200, 'LANG' : 200, 'CHLD' : 200, 'MAAS' : 200, 'UNMR' : 200, 'BLND' : 200, 'DEAF' : 200, 'EXST' : 200, 'MEAL' : 200, 'NSST' : 200, 'NRPS' : 200,  }
loyalty_cost = {'Platinum' : 2000, 'Gold' : 2000, 'Silver' : 2000, 'none':0 }

In [49]:
tag_columns = ['SPECIAL_NAME_CD2', 'SSR_CODE_CD1']

In [50]:
#generating departure key from input

schedule['FlightNumber']=schedule['FlightNumber'].astype(int)
selected_rows = schedule[schedule['ScheduleID'] == specific_schedule_id][['ScheduleID', 'CarrierCode', 'ArrivalAirport', 'DepartureAirport', 'FlightNumber']]
selected_rows['FlightNumber'] = selected_rows['FlightNumber'].astype(str)
selected_rows['Concatenated'] = selected_rows[['DepartureAirport', 'ArrivalAirport', 'FlightNumber']].astype(str).apply(lambda x: ''.join(x), axis=1)
selected_rows['Concatenated'] = specific_date+selected_rows['Concatenated'].replace(' ', '', regex=True)
selected_rows['Concatenated'] = selected_rows[['CarrierCode','Concatenated']].astype(str).apply(lambda x: ''.join(x), axis=1)

#specific departure key is generated and stored in variable specific_dep_key
specific_dep_key= selected_rows.at[selected_rows.index[0], "Concatenated"]
flight_details["DEP_KEY"]=flight_details["DEP_KEY"].astype(str)
# specific_dep_key

In [51]:
# filtering the flights based on departure key generated

filtered_f= flight_details[flight_details["DEP_KEY"]== specific_dep_key]
filtered_f_copy= filtered_f

# filtered_f_copy

In [ ]:
# calcultating the segment scores and pax_cnt scores for reclocs (grouped passengers)

filtered_f_copy['SEG_TOTAL_MULTIPLIED'] = filtered_f_copy['SEG_TOTAL'] * external_multiplier_seg
filtered_f_copy['PAX_CNT_MULTIPLIED'] = filtered_f_copy['PAX_CNT'] * external_multiplier_pax

# Create a new column with the sum of SEG_SEQ_MULTIPLIED and PAX_CNT_MULTIPLIED
filtered_f_copy['SUM_SEG_PAX'] = filtered_f_copy['SEG_TOTAL_MULTIPLIED'] + filtered_f_copy['PAX_CNT_MULTIPLIED']

In [ ]:
# calculating ssr score and loyalty score for each passenger

reclocs_list = filtered_f["RECLOC"].tolist()
filtered_passengers_list = passenger[passenger["RECLOC"].isin(reclocs_list)]
filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
filtered_passengers_list['Loyalty_score'] = filtered_passengers_list['TierLevel'].map(loyalty_cost).fillna(loyalty_cost['none'])

filtered_passengers_list['Total_score'] = filtered_passengers_list['Loyalty_score'] + filtered_passengers_list['ssr_score']
sorted_passengers=filtered_passengers_list.sort_values(by="Total_score", ascending=False)

# filtered_passengers_list
# sorted_passengers

In [54]:
# merging the ssr score and loyalty score generated individually for passenger into the recloc table (grouped passengers)

merged_sorted_passengers = sorted_passengers.groupby('RECLOC')['Total_score'].sum().reset_index()
# merged_sorted_passengers
merged_table = pd.merge(merged_sorted_passengers, filtered_f_copy, on='RECLOC', how='outer')
merged_table['final_score'] = merged_table['Total_score'] + merged_table['SUM_SEG_PAX']
merged_table=merged_table.drop(['Total_score', 'SEG_TOTAL_MULTIPLIED', 'PAX_CNT_MULTIPLIED' ,'SUM_SEG_PAX'], axis=1)
merge_sorted_table = merged_table.sort_values('final_score', ascending=False)

# merge_sorted_table

In [58]:
# generating required lists

final_recloc_list = merge_sorted_table['RECLOC'].tolist()
final_pax_cnt=merge_sorted_table['PAX_CNT'].tolist()
final_seg_seq_cnt=merge_sorted_table['SEG_SEQ'].tolist()
final_score=merge_sorted_table['final_score'].tolist()


# final_score
